# Audio model evaluation

In [2]:
import os
import pickle
import pandas as pd
from pyAudioAnalysis import audioTrainTest as aT

labels = pd.read_csv("project_data/labels.csv", usecols=[0,1], dtype={"id": int, "label": str}, index_col=0)

audio_results = "audio_predictions/"
def get_label_by_story(subject, story):
    return labels.loc[subject].values[0][story-1]

def lou_cross_validation_audio(audio_path, save_results=None):
    save_results = audio_path if save_results==None else save_results
    all_files = []
    for root, dirs, files in os.walk(audio_path):
        for f in files:
            all_files.append(os.path.join(root, f))
    predictions = {}
    for trfile in all_files:
        originalname = trfile
        inter = originalname.split("/")[-1].strip("sub").strip(".wav").split("_")
        sub, st = int(inter[0]), int(inter[1])
        movedname = originalname.replace(originalname.split("/")[1], "fold")
        os.rename(originalname, movedname)
        aT.featureAndTrain([audio_path + "/lie", audio_path + "/truth"], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "audio_models/svm/svm", True)
        prediction = aT.fileClassification(movedname, "audio_models/svm/svm", "svm")[1][1]
        prediction = 1 if prediction>0.5 else 0
        predictions[(sub, st)] = [prediction, get_label_by_story(sub, st)]
        os.rename(movedname, originalname)

    with open(audio_results + save_results + ".pickle", "wb") as f:
        pickle.dump(predictions, f)

control_group = "audio_control_group"
test_group = "audio_test_group"
complete_group = "audio_complete_group"
lou_cross_validation_audio(control_group)
lou_cross_validation_audio(test_group)
lou_cross_validation_audio(complete_group)

TypeError: 'int' object has no attribute '__getitem__'

In [20]:
import pickle
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error as mse

def deserialize(f):
    with open(f, "rb") as ser:
        return pickle.load(ser)


def evaluate_predictions(root):
    for root, dirs, files in os.walk(root):
        for f in files:
            predictions = deserialize(os.path.join(root, f))
            results = np.array(predictions.values()).astype(int)
        #     results mse(results[:, 0], results[:, 1])
#             results = results[results[:,1]==0]
#             print results
            
            results = results[:, 0] - results[:, 1]
            results = (len(results) - sum(abs(results)))/float(len(results))
            f = f.split(".")[0]
            print "Correct classification rate of {}: {:.4f}%".format(f, results*100)

evaluate_predictions("audio_predictions")
print
evaluate_predictions("thermal_predictions")

Correct classification rate of audio_control_group: 77.7778%
Correct classification rate of audio_test_group: 72.7273%
Correct classification rate of audio_complete_group: 62.3188%

Correct classification rate of knn_control: 27.7778%
Correct classification rate of knn_test: 26.6667%
Correct classification rate of knn_complete: 28.7879%
Correct classification rate of lc_control: 38.8889%
Correct classification rate of lc_test: 46.6667%


# Thermal model evaluation

In [ ]:
import pickle 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
import os

thermal_predictions = "thermal_predictions/"

def deserialize(f):
    with open(f, "rb") as ser:
        return pickle.load(ser)

def cross_validate(data, fname, method='knn'):
    preds = {}
    for key in data.keys():
        test = (key, data[key])
        del data[key]
        train = np.vstack(data.values())
        nclass.fit(np.vstack(train[:,0]), train[:,1].astype(int))
        knnpred = nclass.predict(np.vstack(test[1][:,0]))
        truth, lie = sum(knnpred), len(knnpred)-sum(knnpred)
        p = 1 if truth > lie else 0
        preds[key] = [p, test[1][0][1]]
        data[key] = test[1]
    with open(os.path.join(thermal_predictions, method, method + "_" + fname+".pickle"), "wb") as f:
        pickle.dump(preds, f)

complete_group = {}
control_group = deserialize("thermal_data/control_group.pickle")
test_group = deserialize("thermal_data/test_group.pickle")
complete_group.update(control_group)
complete_group.update(test_group)
